# Lab 2: Conditional probability, Bayes' rule and simulating simple games

Please begin by running the code in the following cell to import the packages that are used in this notebook.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats as st
print ("Modules Imported!")

Modules Imported!


### **Part 1: Conditional Probability**

**Conditional probability** is the probability of an event occurring, given that another event has already occurred. The conditional probability of event A given event B is denoted as $P(A|B)$ and is defined as:

$P(A|B) = \frac{P(A \cap B)}{P(B)}$, assuming $P(B) > 0$.

Intuitively, this formula represents a change in the sample space. When we know that event B has occurred, our sample space effectively shrinks from $\Omega$ to the set of outcomes in B. The probability of A happening within this new, smaller sample space is the ratio of outcomes where both A and B occur to the total number of outcomes in B.

For equally likely outcomes, the formula simplifies to counting the elements in the sets:

$P(A|B) = \frac{|A \cap B|}{|B|}$

We can use Python sets to easily calculate this. For example, in our two-dice roll scenario, what is the probability that the sum is exactly 5 (Event A), given that the first die is a 2 (Event D)?

In [2]:
# Example of calculating conditional probability
# Assumes all_outcomes from Problem 2 is available
all_outcomes = {(d1, d2) for d1 in range(1, 7) for d2 in range(1, 7)}
sample_space_size = len(all_outcomes)

# Event A: Sum is 5
event_A = {outcome for outcome in all_outcomes if outcome[0] + outcome[1] == 5}

# Event D: First die is a 2
event_D = {outcome for outcome in all_outcomes if outcome[0] == 2}

# Intersection of A and D
intersection_AD = event_A.intersection(event_D)

# Calculate conditional probability P(A|D) = |A ∩ D| / |D|
prob_A_given_D = len(intersection_AD) / len(event_D)

print(f"Event A (sum is 5): {event_A}")
print(f"Event D (first die is 2): {event_D}")
print(f"Intersection (A ∩ D): {intersection_AD}")
print(f"The probability of the sum being 5 given the first die is 2 is: {prob_A_given_D}")

Event A (sum is 5): {(2, 3), (3, 2), (4, 1), (1, 4)}
Event D (first die is 2): {(2, 4), (2, 1), (2, 3), (2, 6), (2, 2), (2, 5)}
Intersection (A ∩ D): {(2, 3)}
The probability of the sum being 5 given the first die is 2 is: 0.16666666666666666


<br>

**<font color='black'>Problem 1:</font>**
Consider an urn containing 13 numbered balls: 5 are red and 8 are blue.
* The red balls are numbered 1, 2, 3, 4, 5.
* The blue balls are numbered 1, 2, 3, 4, 5, 6, 7, 8.

You draw one ball at random from the urn.

1.  **Create the Sample Space:** Generate a set named `omega_urn` that represents the 13 balls. A good representation is a string, e.g., `'R1'` for red ball 1, `'B2'` for blue ball 2.
2.  **Define Events:** Programmatically create the following event sets:
    * `event_red`: The ball drawn is red.
    * `event_even`: The number on the ball is even.
3.  **Calculate Intersection:** Find the intersection set, `event_red_and_even`.
4.  **Calculate Conditional Probabilities:**
    * Calculate the probability that the ball is red, given that its number is even, $P(\text{Red}|\text{Even})$.
    * Calculate the probability that the ball's number is even, given that the ball is red, $P(\text{Even}|\text{Red})$.

In [3]:
########Student Answer##############
# For Question 1
red_balls = {f'R{i}' for i in range(1, 6)}
blue_balls = {f'B{i}' for i in range(1, 9)}
omega_urn = red_balls.union(blue_balls)
print(f"Sample Space: {omega_urn}")

# For Question 2
event_red = {ball for ball in omega_urn if ball.startswith('R')}
event_even = {ball for ball in omega_urn if int(ball[1:]) % 2 == 0}
print(f"Event Red: {event_red}")
print(f"Event Even: {event_even}")

# For Question 3
event_red_and_even = event_red.intersection(event_even)
print(f"Event Red and Even: {event_red_and_even}")

# For Question 4
prob_red_given_even = len(event_red_and_even) / len(event_even)
prob_even_given_red = len(event_red_and_even) / len(event_red)
print(f"P(Red|Even): {prob_red_given_even}")
print(f"P(Even|Red): {prob_even_given_red}")


Sample Space: {'R1', 'R5', 'B5', 'B3', 'B6', 'B1', 'B4', 'R2', 'B8', 'R3', 'B7', 'R4', 'B2'}
Event Red: {'R1', 'R5', 'R2', 'R3', 'R4'}
Event Even: {'B6', 'B4', 'R2', 'B8', 'R4', 'B2'}
Event Red and Even: {'R4', 'R2'}
P(Red|Even): 0.3333333333333333
P(Even|Red): 0.4


__Answer:__ (Your answer here)

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 1</SPAN>**

### **Part 2: Bayes' Theorem**

**Bayes' Theorem** is a cornerstone of probability theory that allows us to update our beliefs about an event based on new evidence. It relates the conditional probabilities of two events. The theorem is stated as:

$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$

This theorem is incredibly useful because it lets us find $P(A|B)$ when we might only know $P(B|A)$, along with the individual probabilities $P(A)$ and $P(B)$. It provides a way to "invert" the conditioning. To use it, we need to calculate three components: $P(B|A)$, the prior probability $P(A)$, and the marginal probability $P(B)$.

Below is a code example demonstrating how to calculate these components and apply the theorem for a two-dice roll scenario. Let A be the event "the sum is 7" and B be "the first die is 4". We want to find $P(A|B)$.

In [4]:
# Example of applying Bayes' Theorem
all_outcomes = {(d1, d2) for d1 in range(1, 7) for d2 in range(1, 7)}
sample_space_size = len(all_outcomes)

# Define events
event_A = {outcome for outcome in all_outcomes if outcome[0] + outcome[1] == 7} # Sum is 7
event_B = {outcome for outcome in all_outcomes if outcome[0] == 4} # First die is 4

# Calculate prior and marginal probabilities P(A) and P(B)
prob_A = len(event_A) / sample_space_size
prob_B = len(event_B) / sample_space_size

# Calculate P(B|A): The probability the first die is 4 GIVEN the sum is 7.
# Our sample space is reduced to event_A.
intersection_AB = event_A.intersection(event_B)
prob_B_given_A = len(intersection_AB) / len(event_A)

# Apply Bayes' Theorem
# Check if P(B) is non-zero to avoid division by zero
if prob_B > 0:
    prob_A_given_B = (prob_B_given_A * prob_A) / prob_B
    print(f"P(A) = {prob_A:.4f}")
    print(f"P(B) = {prob_B:.4f}")
    print(f"P(B|A) = {prob_B_given_A:.4f}")
    print(f"P(A|B) calculated via Bayes' Theorem is: {prob_A_given_B:.4f}")
else:
    print("P(B) is zero, cannot calculate conditional probability.")

P(A) = 0.1667
P(B) = 0.1667
P(B|A) = 0.1667
P(A|B) calculated via Bayes' Theorem is: 0.1667


<br>

**<font color='black'>Problem 2:</font>**
A friend rolls two standard six-sided dice but hides the result. They then look at the dice and tell you, "At least one of the dice is a 6." Your task is to determine the probability that the sum of the two dice is exactly 9, given this new information.

**Events:**
* Event A: The sum of the dice is 9.
* Event B: At least one die is a 6.

**Tasks:**
1.  **Generate Sets & Probabilities:** Programmatically generate the sets for `event_A` and `event_B`. Then, calculate the three probabilities needed for Bayes' Theorem: $P(A)$, $P(B)$, and $P(B|A)$.
    *Hint: To find $P(B|A)$, consider the reduced sample space of `event_A` and see which outcomes also satisfy event B.*
2.  **Apply Bayes' Theorem:** Use the probabilities from Task 1 to calculate $P(A|B)$.
3.  **Verification:** For verification, calculate $P(A|B)$ directly using the definition of conditional probability ($P(A|B) = \frac{|A \cap B|}{|B|}$) and show that your result matches the answer from Task 2.

In [5]:
########Student Answer##############
event_A = {outcome for outcome in all_outcomes if outcome[0]+outcome[1] == 9}
event_B = {outcome for outcome in all_outcomes if outcome[0] == 6 or outcome[1] == 6}

prob_A = len(event_A) / sample_space_size
prob_B = len(event_B) / sample_space_size

intersection_AB = event_A.intersection(event_B)
prob_B_given_A = len(intersection_AB) / len(event_A)

print(f"Event A: {event_A}")
print(f"Event B: {event_B}")
print(f"P(A) = {prob_A:.4f}")
print(f"P(B) = {prob_B:.4f}")
print(f"P(B|A) = {prob_B_given_A:.4f}")

prob_A_given_B = (prob_B_given_A * prob_A) / prob_B
print(f"P(A|B) calculated via Bayes' Theorem is: {prob_A_given_B:.4f}")

prob_A_given_B_direct = len(intersection_AB) / len(event_B)
print(f"P(A|B) calculated directly is: {prob_A_given_B_direct:.4f}")

assert abs(prob_A_given_B - prob_A_given_B_direct) < 1e-6, "The two methods should yield the same result."
print("Both methods yield the same result.")

Event A: {(4, 5), (5, 4), (6, 3), (3, 6)}
Event B: {(6, 2), (6, 5), (6, 1), (4, 6), (6, 4), (2, 6), (5, 6), (3, 6), (6, 6), (1, 6), (6, 3)}
P(A) = 0.1111
P(B) = 0.3056
P(B|A) = 0.5000
P(A|B) calculated via Bayes' Theorem is: 0.1818
P(A|B) calculated directly is: 0.1818
Both methods yield the same result.


**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 2</SPAN>**

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">Problem 3:</SPAN>** 
Instead of playing cards, let's analyze the probabilities of rolling certain combinations with five standard 6-sided dice, similar to hands in the game Yahtzee.

<ol><li>Calculate the theoretical probabilities of rolling a THREE OF A KIND, a FULL HOUSE, a FOUR OF A KIND, and a LARGE STRAIGHT. Print out these probabilities. You need to write down your calculation process, not just the final results.
<ul>
    <li>A <strong>THREE OF A KIND</strong> consists of three dice showing the same number, and the other two dice showing two different numbers (e.g., 4, 4, 4, 1, 2).</li>
    <li>A <strong>FULL HOUSE</strong> consists of three dice of one number and two dice of another (e.g., 5, 5, 5, 2, 2).</li>
    <li>A <strong>FOUR OF A KIND</strong> consists of four dice showing the same number and one die showing another number (e.g., 1, 1, 1, 1, 6).</li>
    <li>A <strong>LARGE STRAIGHT</strong> is a sequence of 5 numbers (i.e., 1-2-3-4-5 or 2-3-4-5-6).</li>
</ul>
</li> 
<li>Simulate 1,000,000 rolls of five dice. Count the number of times you get each of the hands above and find their empirical probabilities.</li>
<li>Do the theoretical and empirical probabilities match up relatively well?</li>
</ol>

In [ ]:
########Student Answer##############
import math
from math import comb
total_outcomes = 6**5
# THREE OF A KIND
# Choose 1 number for three dice: C(6,1)
# Choose 2 different numbers for the other two dice: C(5,2)
# Arrange the 5 dice with the chosen numbers: C(5,3) * 2!
ways_three_of_a_kind = comb(6,1) * comb(5,2) * comb(5,3) * math.factorial(2)
prob_three_of_a_kind = ways_three_of_a_kind / total_outcomes
print(f"Probability of Three of a Kind: {prob_three_of_a_kind:.6f}")

# FULL HOUSE
# Choose 1 number for three dice: C(6,1)
# Choose 1 number for the other two dice: C(5,1)
# Arrange the 5 dice with the chosen numbers: C(5,3)
ways_full_house = comb(6,1) * comb(5,1) * comb(5,3)
prob_full_house = ways_full_house / total_outcomes
print(f"Probability of Full House: {prob_full_house:.6f}")

# FOUR OF A KIND
# Choose 1 number for four dice: C(6,1)
# Choose 1 number for the other die: C(5,1)
# Arrange the 5 dice with the chosen numbers: C(5,4)
ways_four_of_a_kind = comb(6,1) * comb(5,1) * comb(5,4)
prob_four_of_a_kind = ways_four_of_a_kind / total_outcomes
print(f"Probability of Four of a Kind: {prob_four_of_a_kind:.6f}")

# A LARGE STRAIGHT
# 2 possible sequences: {1,2,3,4,5} and {2,3,4,5,6}
# Each sequence can be arranged in 5! ways
ways_large_straight = 2 * math.factorial(5)
prob_large_straight = ways_large_straight / total_outcomes
print(f"Probability of a Large Straight: {prob_large_straight:.6f}")

num_simulations = 1000000
counts = {
    "THREE OF A KIND": 0,
    "FULL HOUSE": 0,
    "FOUR OF A KIND": 0,
    "LARGE STRAIGHT": 0
}

for _ in range(num_simulations):
    rolls = np.random.randint(1, 7, size=5)
    unique, counts_rolls = np.unique(rolls, return_counts=True)
    
    if sorted(counts_rolls) == [1, 1, 3]:
        counts["THREE OF A KIND"] += 1
    elif sorted(counts_rolls) == [2, 3]:
        counts["FULL HOUSE"] += 1
    elif sorted(counts_rolls) == [1, 4]:
        counts["FOUR OF A KIND"] += 1
    elif len(unique) == 5 and (max(unique) - min(unique) == 4):
        counts["LARGE STRAIGHT"] += 1

for hand, count in counts.items():
    empirical_prob = count / num_simulations
    print(f"Empirical Probability of {hand}: {empirical_prob:.6f}")
    if hand == "THREE OF A KIND":
        assert abs(empirical_prob - prob_three_of_a_kind) < 0.01, "Empirical probability deviates significantly from theoretical probability."
    elif hand == "FULL HOUSE":
        assert abs(empirical_prob - prob_full_house) < 0.01, "Empirical probability deviates significantly from theoretical probability."
    elif hand == "FOUR OF A KIND":
        assert abs(empirical_prob - prob_four_of_a_kind) < 0.01, "Empirical probability deviates significantly from theoretical probability."
    elif hand == "LARGE STRAIGHT":
        assert abs(empirical_prob - prob_large_straight) < 0.01, "Empirical probability deviates significantly from theoretical probability."

print("Empirical probabilities are consistent with theoretical probabilities.")

Probability of Three of a Kind: 0.154321
Probability of Full House: 0.038580
Probability of Four of a Kind: 0.019290
Probability of a Large Straight: 0.030864
Empirical Probability of THREE OF A KIND: 0.153977
Empirical Probability of FULL HOUSE: 0.038800
Empirical Probability of FOUR OF A KIND: 0.019187
Empirical Probability of LARGE STRAIGHT: 0.031020


**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 3</SPAN>**

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">Problem 4:</SPAN>** A classic problem when being introduced to probability is the Monty Hall problem. If you've ever seen "Let's Make a Deal" on television, this problem takes from that show. You're the contestant. The host of the show gives you three doors to choose from. Now the situation has upgraded. We have four doors , one door chosen at random holds a grand prize and the other three hold worthless items. You choose your door, and then the host reveals one of the doors you didn't choose such that it always holds a worthless item.  (If you initially choose the door with the grand prize, the host reveals either of the other doors with equal probability.) So now there are three doors left and the host asks you whether you would like to switch. What should you do?
<ol><li>Write down your first reaction? Would you switch doors or keep the one you have? Why?</li>
    <li>Create this scenario and simulate the strategy of sticking with the same door 1,000,000 times. What percentage of time did you win?</li>
    <li>Simulate the strategy of switching doors 1,000,000 times. What percentage of time did you win?</li>
    <li>Which strategy would you use now? Explain why this is the case.</li>
</ol>

In the simulation, you can simulate every step that happens in the actual game and exactly as it happens, even for very trivial steps. You can also clever-guess some parts and skip some trivial steps; if you do that, make sure you briefly reason about why you do so in comments, so that the graders know that you understand what actually happens in the game. There is no timing requirement for this problem.  

In [ ]:
########Student Answer##############
# For Question 1: My first reaction is switch the doors. 
# The probability that I chose the right door is 1/4.
# When the host opens a worthless door, the 3/4 probability
# is concentrated on the other two remaining doors.
# So the probability is higher than the initial 1/4.

num_simulations == 100000
doors = {1, 2, 3, 4}

# For Question 2
stick_wins = 0
for i in range(num_simulations):
    prize_door = np.random.randint(1, 5)
    player_choice = np.random.randint(1, 5)
    
    if player_choice == prize_door:
        stick_wins += 1
prob_stick = stick_wins / num_simulations
print(f"Probability of winning by sticking: {prob_stick:.4f}")

switch_wins = 0
for i in range(num_simulations):
    prize_door = np.random.randint(1, 5)
    player_choice = np.random.randint(1, 5)
    
    remaining_doors = doors - {player_choice}
    if prize_door in remaining_doors:
        remaining_doors.remove(prize_door)
    host_opens = np.random.choice(list(remaining_doors))
    
    switch_options = doors - {player_choice, host_opens}
    player_switch = np.random.choice(list(switch_options))
    
    if player_switch == prize_door:
        switch_wins += 1
prob_switch = switch_wins / num_simulations
print(f"Probability of winning by switching: {prob_switch:.4f}")

"""
The results show that switching doors yields a higher probability of winning the prize 
compared to sticking with the initial choice. 
1. P(Stick_Win) = 0.25
2. P(Switch_Win) = P(First_Wrong) * P(Second_Right|First_Wrong) = 0.75 * 0.5 = 0.375
"""

Probability of winning by sticking: 0.2496
Probability of winning by switching: 0.3748


**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 4</SPAN>**

<div class="alert alert-block alert-warning"> 
## Academic Integrity Statement ##

By submitting the lab with this statement, you declare you have written up the lab entirely by yourself, including both code and markdown cells. You also agree that you should not share your code with anyone else. Any violation of the academic integrity requirement may cause an academic integrity report to be filed that could go into your student record. See <a href="https://provost.illinois.edu/policies/policies/academic-integrity/students-quick-reference-guide-to-academic-integrity/">Students' Quick Reference Guide to Academic Integrity</a> for more information. 